Functions to extract the data presented in Figure 1F

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

def extract_bckgrd(df, type_data):
    
    one = type_data+"_ROI_0"
    two = type_data+"_ROI_1"
    three = type_data+"_ROI_2"
    data = df[[one, two, three]]
    data['ave'] = data.mean(axis=1)
    data['sem'] = data.std(axis=1)/math.sqrt(3)
    
    bckgrd_ave = pd.Series.to_frame(data['ave'])
    bckgrd_sem = pd.Series.to_frame(data['sem'])
    
    return bckgrd_ave, bckgrd_sem

def compile_bckgrd(extract_bckgrd, df, type_data, back_ave, back_sem):
    
    bckgrd_ave, bckgrd_sem = extract_bckgrd(df, type_data)
    
    back_ave.append(bckgrd_ave)
    back_sem.append(bckgrd_sem)
    
    return back_ave, back_sem
    
def extract_aves(df, type_data):

    one = type_data+"_ROI_0"
    two = type_data+"_ROI_1"
    three = type_data+"_ROI_2"
    data = df[[one, two, three]]
    data['ave'] = data.mean(axis=1)
    data['sem'] = data.std(axis=1)/math.sqrt(3)

    ave_col = pd.Series.to_frame(data['ave'])
    initial = ave_col.iloc[0]
    ave_norm_to_initial = ave_col.divide(initial)
    ave_norm = np.asarray(ave_norm_to_initial.as_matrix)

    sem_col = pd.Series.to_frame(data['sem'])

    ave_np = np.asarray(data['ave'])
    sem_np = np.asarray(data['sem'])

    return ave_col, sem_col, ave_norm_to_initial, ave_norm, ave_np, sem_np

def compile_lists(extract_aves, df, type_data, ave, sem, ave_raw,
    sem_raw, ave_norm, ave_norm_np):

    ave_col, sem_col, ave_norm_to_initial, ave_np_norm, ave_np, sem_np = extract_aves(df, type_data)

    ave.append(ave_np)
    sem.append(sem_np)
    
    ave_norm.append(ave_norm_to_initial)
    ave_norm_np.append(ave_np_norm)

    ave_raw.append(ave_col)
    sem_raw.append(sem_col)
    
    return [ave, sem, ave_norm, ave_norm_np, ave_raw, sem_raw]

def finalize_ave_and_sem(ave_raw, ave_norm, curr_path, save_as):

    ave_norm_df = pd.concat(ave_norm, axis=1)

    df_dimensions = ave_norm_df.shape
    num_cells = df_dimensions[1]

    ave_norm_df['Final Ave'] = ave_norm_df.mean(axis=1)
    ave_norm_df['SEM'] = (ave_norm_df.std(axis=1))/math.sqrt(num_cells)
    
    save_path = os.path.join(curr_path, save_as)
    ave_norm_df.to_csv(save_path)

    final_norm_ave = np.asarray(ave_norm_df['Final Ave'])
    final_norm_sem = np.asarray(ave_norm_df['SEM'])

    return final_norm_ave, final_norm_sem

def normalize_to_background(ave_raw, bckgrd_ave, curr_path, save_as):
    
    ave_raw_df = pd.concat(ave_raw, axis=1)
    bckgrd_df = pd.concat(bckgrd_ave, axis=1)
    
    df_dimensions = ave_raw_df.shape
    num_cells = df_dimensions[1]
    
    norm_to_bckgrd = ave_raw_df.div(bckgrd_df)
    
    norm_to_bckgrd['Final Ave'] = norm_to_bckgrd.mean(axis=1)
    norm_to_bckgrd['SEM'] = (norm_to_bckgrd.std(axis=1))/math.sqrt(num_cells)
    
    save_path = os.path.join(curr_path, save_as)
    norm_to_bckgrd.to_csv(save_path)

    final_norm_ave = np.asarray(norm_to_bckgrd['Final Ave'])
    final_norm_sem = np.asarray(norm_to_bckgrd['SEM'])

    return final_norm_ave, final_norm_sem

#directory = directory of your choice
count = 0
mean_ave = []
mean_sem = []

mean_ave_raw = []
mean_sem_raw = []

mean_ave_norm = []
mean_ave_norm_np = []

max_ave = []
max_sem = []

max_ave_raw = []
max_sem_raw = []

max_ave_norm = []
max_ave_norm_np = []

back_ave = []
back_sem = []

for root, expt_dirs, files in os.walk(directory):
    for f in files:
        if f == 'intensities.csv' or f == "MotherCell_intensities.csv" or f == "Target_intensities.csv":
            current_file = os.path.join(root, f)
            data = pd.read_csv(current_file)

            compile_lists(extract_aves, data, 'Mean', mean_ave,
                mean_sem, mean_ave_raw, mean_sem_raw, mean_ave_norm,
                mean_ave_norm_np)

            compile_lists(extract_aves, data, 'Max', max_ave,
                max_sem, max_ave_raw, max_sem_raw, max_ave_norm,
                max_ave_norm_np)
        elif f == 'Background_intensities.csv':
            
            current_file = os.path.join(root, f)
            bckgrd = pd.read_csv(current_file)

            back_ave, back_sem = compile_bckgrd(extract_bckgrd, bckgrd, 'Mean', back_ave, back_sem)


final_mean_norm_ave, final_mean_norm_sem = normalize_to_background(mean_ave_raw, back_ave, directory, 'Mean_Intensities_Background.csv')
final_max_norm_ave, final_max_norm_sem = normalize_to_background(max_ave_raw, max_ave_norm, directory, 'Max_Intensities_Background.csv')

print "Done"

In [ ]:
def finalize_ave_and_sem(ave_raw, ave_norm, curr_path, save_as):

    #ave_raw_df = pd.concat([x[0] for x in ave_raw], axis=1)
    #ave_norm_df = pd.concat([x[0] for x in ave_norm], axis=1)
    ave_raw_df = pd.concat([x['ave'] for x in ave_raw], axis=1)
    ave_norm_df = pd.concat([x['ave'] for x in ave_norm], axis=1)

    df_dimensions = ave_raw_df.shape
    num_cells = df_dimensions[1]

    ave_norm_df['Final Ave'] = ave_norm_df.mean(axis=1)
    ave_norm_df['SEM'] = (ave_norm_df.std(axis=1))/math.sqrt(num_cells)
    
    save_path = os.path.join(curr_path, save_as)
    ave_norm_df.to_csv(save_path)

    final_norm_ave = np.asarray(ave_norm_df['Final Ave'])
    final_norm_sem = np.asarray(ave_norm_df['SEM'])

    return final_norm_ave, final_norm_sem

def compile_lists(extract_aves, df, type_data, time, be, ave_raw,
    sem_raw, ave_normalized):

    ave_col, sem_col, ave_norm = extract_aves(df, type_data, time, be)
    
    ave_normalized.append(ave_norm)

    ave_raw.append(ave_col)
    sem_raw.append(sem_col)
    
    return [ave_raw, sem_raw, ave_normalized]

def extract_aves(df, type_data, time, be):

    one = type_data+"_ROI_0"
    two = type_data+"_ROI_1"
    three = type_data+"_ROI_2"
    data = df[[one, two, three]]
    data['ave'] = data.mean(axis=1)
    data['sem'] = data.std(axis=1)/math.sqrt(3)

    ave_col = pd.Series.to_frame(data['ave']) 
    initial = ave_col.iloc[0]
    ave_norm = ave_col.divide(initial) 
    sem_col = pd.Series.to_frame(data['sem']) 

    slice_ave_col = slice_timepoints(ave_col, time, be)
    slice_ave_norm = slice_timepoints(ave_norm, time, be)
    slice_sem_col = slice_timepoints(sem_col, time, be)
    
    return slice_ave_col, slice_sem_col, slice_ave_norm

def extract_bckgrd_aves(df, type_data, time, be):

    one = type_data+"_ROI_0"
    two = type_data+"_ROI_1"
    three = type_data+"_ROI_2"
    data = df[[one, two, three]]
    data['ave'] = data.mean(axis=1)
    data['sem'] = data.std(axis=1)/math.sqrt(3)

    ave_col = pd.Series.to_frame(data['ave']) 
    #initial = ave_col.iloc[0]
    #ave_norm = ave_col.divide(initial) 
    sem_col = pd.Series.to_frame(data['sem']) 

    slice_ave_col = slice_timepoints(ave_col, time, be)
    #slice_ave_norm = slice_timepoints(ave_norm, time, be)
    slice_sem_col = slice_timepoints(sem_col, time, be)
    
    return slice_ave_col, slice_sem_col

def compile_bckgrd_be(extract_bckgrd_aves, df, type_data, time, be, back_ave, back_sem):
    
    bckgrd_ave, bckgrd_sem = extract_bckgrd_aves(df, type_data, time, be)
    
    back_ave.append(bckgrd_ave)
    back_sem.append(bckgrd_sem)
    
    return back_ave, back_sem
    
def extract_BE_time(be_text):

    with open(be_text, 'r') as be:

        reader = be.readlines()

        all_lines = []
        for line in reader:
            split = line.split("\t")
            all_lines.append(split)

        be = int(all_lines[1][1])

    return be

def slice_timepoints(df, timepoints, be):

    labels = [x for x in range(62)]
    intensity_be_index = 0
    start = be - 41
    if start < 0:
        start = start - start
    end = be + 20
    if end > len(timepoints):
        end = len(timepoints)

    sliced = df[start:end]

    time_sliced = timepoints[start:end]

    dex_be = time_sliced.index(be)

    if sliced.shape[0] < 61:
        i = 0
        if dex_be < 40:
            num_zeroes_to_pad = 40 - dex_be
            zeroes = np.asarray([1.0]*num_zeroes_to_pad)
            column_name = list(df.columns.values)[0]
            #print column_name
            zeroes_df = pd.DataFrame(zeroes, columns=[column_name])
            df_to_concat = [zeroes_df, sliced]
            insert_zeroes = pd.concat(df_to_concat).reset_index().reindex(labels)

        elif dex_be == 40:
            num_zeroes_to_pad = int(61 - sliced.shape[0])
            zeroes = np.asarray([1.0]*num_zeroes_to_pad)
            column_name = list(df.columns.values)[0]
            #print column_name
            zeroes_df = pd.DataFrame(zeroes, columns=[column_name])
            df_to_concat = [sliced, zeroes_df]
            insert_zeroes = pd.concat(df_to_concat).reset_index().reindex(labels)        
    else:
        insert_zeroes = sliced.reset_index().reindex(labels)
        
    #print insert_zeroes

    return insert_zeroes

def normalize_to_background(ave_raw, bckgrd_ave, curr_path, save_as):
    
    ave_raw_df = pd.concat(ave_raw, axis=1)
    bckgrd_df = pd.concat(bckgrd_ave, axis=1)
    
    df_dimensions = ave_raw_df.shape
    num_cells = df_dimensions[1]
    
    norm_to_bckgrd = ave_raw_df.div(bckgrd_df)
    
    del norm_to_bckgrd['index']
    
    norm_to_bckgrd['Final Ave'] = norm_to_bckgrd.mean(axis=1)
    norm_to_bckgrd['SEM'] = (norm_to_bckgrd.std(axis=1))/math.sqrt(num_cells)
    
    save_path = os.path.join(curr_path, save_as)
    norm_to_bckgrd.to_csv(save_path)

    final_norm_ave = np.asarray(norm_to_bckgrd['Final Ave'])
    final_norm_sem = np.asarray(norm_to_bckgrd['SEM'])

    return final_norm_ave, final_norm_sem

directory = '/Users/Kristen/Desktop/12132016/PULSED_POLARIZATION/WYK8308_Pulsed_Polarization/WYK8308_3Pulse_Pol'
count = 0

mean_ave_raw = []
mean_sem_raw = []
mean_ave_norm = []

back_ave = []
back_sem = []

for root, expt_dirs, files in os.walk(directory):
    for f in files:
        if f == 'intensities.csv' or f == "MotherCell_intensities.csv" or f == "Target_intensities.csv":
                current_file = os.path.join(root, f)
                data = pd.read_csv(current_file)
        
                num_timepoints = data.shape[0]
                time = range(1, num_timepoints+1)

                bud_emerge_path = os.path.join(root, 'BudEmergence.txt')
                be = extract_BE_time(bud_emerge_path)

                all_mean_lists = compile_lists(extract_aves, data, 'Mean', time, be, mean_ave_raw, mean_sem_raw, mean_ave_norm)
                all_max_lists = compile_lists(extract_aves, data, 'Max', time, be, max_ave_raw, max_sem_raw, max_ave_norm)
        elif f == 'Background_intensities.csv':
            #mother_cell_path = os.path.join(root, 'MotherCell')
            #if os.path.isdir(mother_cell_path):
                current_file = os.path.join(root, f)
                bckgrd = pd.read_csv(current_file)
            
                num_timepoints = bckgrd.shape[0]
                time = range(1, num_timepoints+1)

                #bud_emerge_path = os.path.join(mother_cell_path, 'BudEmergence.txt')
                bud_emerge_path = os.path.join(root, 'BudEmergence.txt')
                print bud_emerge_path
                be = extract_BE_time(bud_emerge_path)

                back_ave, back_sem = compile_bckgrd_be(extract_bckgrd_aves, bckgrd, 'Mean', time, be, back_ave, back_sem)

final_mean_norm_ave, final_mean_norm_sem = finalize_ave_and_sem(mean_ave_raw, mean_ave_norm, directory, 'Mean_Polarization_Background_Intensities.csv')
final_max_norm_ave, final_max_norm_sem = finalize_ave_and_sem(max_ave_raw, max_ave_norm, directory, 'Max_Polarization_Intensities.csv')
print 'Done'

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt


def setBoxColors(bp, color):
    
    for item in bp:
        curr_property = bp[item]
        for prop in curr_property:
            plt.setp(prop, color=color)

directory = '/Users/Kristen/Desktop/12132016/PULSED_POLARIZATION/WYK8440_Pulsed_Polarization'
all_directories = [name for name in os.listdir(directory)]

extracted_data = {}

for name in all_directories:
    curr_dir = os.path.join(directory, name)
    if (name != ".DS_Store"):
        for f in os.listdir(curr_dir):
            if f == "Mean_Polarization_Background_Intensities.csv":
                current_file = os.path.join(curr_dir, f)
                data = pd.read_csv(current_file)
                ten_min_slice = data.loc[[30],:]
                ten_min_slice.drop('SEM', axis=1, inplace=True)
                ten_min_slice.drop('Final Ave', axis=1, inplace=True)
                ten_min_slice.drop('Unnamed: 0', axis=1, inplace=True)
                measurements = np.asarray(ten_min_slice)
                extracted_data[name] = measurements[0]
    

bem1_names = ['WYK8308_Control_Pol','WYK8308_001Pulse_Pol','WYK8308_1Pulse_Pol', 'WYK8308_2Pulse_Pol', 'WYK8308_3Pulse_Pol', 'WYK8308_30Sec_Pol','WYK8308_15Sec_Pol', 'WYK8308_10Sec_Pol']
cdc24_names = ['WYK8440_Control_Pol','WYK8440_001Pulse_Pol','WYK8440_1Pulse_Pol', 'WYK8440_2Pulse_Pol', 'WYK8440_3Pulse_Pol', 'WYK8440_30Sec_Pol','WYK8440_15Sec_Pol', 'WYK8440_10Sec_Pol']

plt.figure()
cdc24_data_nonpol = []
for curr_data in cdc24_names:
    data = extracted_data[curr_data]
    cdc24_data_nonpol.append(data)
bp = plt.boxplot(cdc24_data_nonpol)
setBoxColors(bp, '#8470ff')
plt.ylim(0.8,1.4)
#save_to = os.path.join(directory, Whatever you want)
print(save_to)
#plt.savefig(save_to)

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt


def setBoxColors(bp, color):
    
    for item in bp:
        curr_property = bp[item]
        for prop in curr_property:
            plt.setp(prop, color=color)

directory = '/Users/Kristen/Desktop/12132016/PULSED_POLARIZATION/WYK8440_Pulsed_Polarization'
all_directories = [name for name in os.listdir(directory)]
all_directories = all_directories[1:]

extracted_data = {}

for name in all_directories:
    curr_dir = os.path.join(directory, name)
    for f in os.listdir(curr_dir):
        if f == "Mean_Intensities_Background.csv":
            current_file = os.path.join(curr_dir, f)
            data = pd.read_csv(current_file)
            ten_min_slice = data.loc[[10],:]
            ten_min_slice.drop('SEM', axis=1, inplace=True)
            ten_min_slice.drop('Final Ave', axis=1, inplace=True)
            ten_min_slice.drop('Unnamed: 0', axis=1, inplace=True)
            measurements = np.asarray(ten_min_slice)
            extracted_data[name] = measurements[0]
    
bem1_names = ['WYK8308_Control_Pol','WYK8308_001Pulse_Pol','WYK8308_1Pulse_Pol', 'WYK8308_2Pulse_Pol', 'WYK8308_3Pulse_Pol', 'WYK8308_30Sec_Pol','WYK8308_15Sec_Pol', 'WYK8308_10Sec_Pol']
cdc24_names = ['WYK8440_Control_Pol','WYK8440_001Pulse_Pol','WYK8440_1Pulse_Pol', 'WYK8440_2Pulse_Pol', 'WYK8440_3Pulse_Pol', 'WYK8440_30Sec_Pol','WYK8440_15Sec_Pol', 'WYK8440_10Sec_Pol']

print(extracted_data.keys())

plt.figure()
cdc24_data_pol = []
for curr_data in cdc24_names:
    data = extracted_data[curr_data]
    cdc24_data_pol.append(data)
bp = plt.boxplot(cdc24_data_pol)
setBoxColors(bp, '#ff8c00')
plt.ylim(0.8,1.4)

#save_to = os.path.join(directory, "cdc24_Boxplot_Polarized_Background.pdf")
print(save_to)
#plt.savefig(save_to)

In [ ]:
ax = plt.axes()
print len(cdc24_data_pol)
print len(cdc24_data_nonpol)

flierprops = dict(marker='x', markerfacecolor='black', markersize=3,linestyle='none')

bem1_pol = plt.boxplot(bem1_data_pol, positions = [1,4,7,10,13,16,19,22], flierprops=flierprops)
setBoxColors(bem1_pol, '#778899')
bem1_nonpol = plt.boxplot(bem1_data_nonpol, positions = [2,5,8,11,14,17,20,23], flierprops=flierprops)
setBoxColors(bem1_nonpol, '#000000')

for i, n in enumerate([1,4,7,10,13,16,19,22]):
    print n
    y = bem1_data_pol[i]
    x = np.random.normal(n, 0.04, size=len(y))
    #plt.plot(x, y, 'r.')
    
for i, n in enumerate([2,5,8,11,14,17,20,23]):
    print n
    y = bem1_data_nonpol[i]
    x = np.random.normal(n, 0.04, size=len(y))
    #plt.plot(x, y, 'r.', )

plt.xlim(0,24)
plt.ylim(0.75,1.4)

d = '/Users/Kristen/Desktop/12132016/PULSED_POLARIZATION'

save_to = os.path.join(d, "bem1_BoxPlot_SmallBuds_All_Pulses_Background.pdf")
print(save_to)
plt.savefig(save_to)